In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src/')

In [2]:
pip install --user tables

Note: you may need to restart the kernel to use updated packages.


## Fridge case

In [3]:
APPLIANCE = ['fridge']
THRESHOLD = [20.]
MIN_ON = [10.]
MIN_OFF = [2.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [4]:
from utils.preprocessing import get_status
ds_meter = []
ds_appliance = []
ds_status = []
houses=[15]
for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/fridge/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [5]:
from utils.preprocessing import chain2filter

houses=[15]
THRESHOLDS=[50,100,150,200,250,300]    
datafilter=[]
for threshold in THRESHOLDS:
    original = []
    for k in houses:
        ds = pd.read_feather('../data/refit/feather_files/fridge/REFIT_%d.feather' %(k))
        ds.set_index('datetime', inplace=True)  
        meter = ds[METER]
        original.append(meter)
    datafilter.append(chain2filter(original, threshold, len(houses)))


ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/fridge/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]

dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False)


In [7]:
from utils.net import PTPNet
from utils.evaluation import print_metrics

###### print no filtered data metrics ######
print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_fridge_network_weights.pth',
              x_test = dl_test,
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER,
              thr = 0.5)
###### print filtered data metrics #######
for i in range(6):
    print('-----------------------------')
    print('Threshold = ' + str(THRESHOLDS[i]))
    ds_house_filter = Power(datafilter[i][-1], ds_appliance[-1], ds_status[-1], SEQ_LEN, BORDER, MAX_POWER, False)
    dl_test_filter = DataLoader(dataset = ds_house_filter, batch_size = BATCH_SIZE, shuffle=False)

    print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_fridge_network_weights.pth',
              x_test = dl_test_filter, 
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER, thr = 0.5)


fridge
F1 score  : 0.801
Precision : 0.810
Recall    : 0.792
Accuracy  : 0.892
MCC       : 0.727
MAE       : 9.37
SAE       : -0.023
-----------------------------
Threshold = 50

fridge
F1 score  : 0.697
Precision : 0.685
Recall    : 0.708
Accuracy  : 0.830
MCC       : 0.579
MAE       : 14.05
SAE       : 0.034
-----------------------------
Threshold = 100

fridge
F1 score  : 0.624
Precision : 0.607
Recall    : 0.643
Accuracy  : 0.787
MCC       : 0.476
MAE       : 17.33
SAE       : 0.060
-----------------------------
Threshold = 150

fridge
F1 score  : 0.625
Precision : 0.610
Recall    : 0.641
Accuracy  : 0.788
MCC       : 0.478
MAE       : 17.21
SAE       : 0.051
-----------------------------
Threshold = 200

fridge
F1 score  : 0.607
Precision : 0.588
Recall    : 0.627
Accuracy  : 0.776
MCC       : 0.451
MAE       : 18.15
SAE       : 0.067
-----------------------------
Threshold = 250

fridge
F1 score  : 0.607
Precision : 0.591
Recall    : 0.625
Accuracy  : 0.777
MCC       : 0.452
MAE

## Dishwasher case

In [8]:
APPLIANCE = ['dishwasher']
THRESHOLD = [30.]
MIN_ON = [1400. / 60]
MIN_OFF = [1600. / 60]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [9]:
from utils.preprocessing import get_status
ds_meter = []
ds_appliance = []
ds_status = []
houses=[2]

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/dishwasher/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [10]:
from utils.preprocessing import chain2filter

houses=[2]
THRESHOLDS=[50,100,150,200,250,300]    
datafilter=[]
for threshold in THRESHOLDS:
    original = []
    for k in houses:
        ds = pd.read_feather('../data/refit/feather_files/dishwasher/REFIT_%d.feather' %k)
        ds.set_index('datetime', inplace=True)  
        meter = ds[METER]
        original.append(meter)
    datafilter.append(chain2filter(original, threshold, len(houses)))

ds_meter = []
ds_appliance = []
ds_status = []

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/dishwasher/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]
dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False)

In [12]:
from utils.net import PTPNet
from utils.evaluation import print_metrics

###### print no filtered data metrics ######
print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_dishwasher_network_weights.pth',
              x_test = dl_test,
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER,
              thr = 0.5)
###### print filtered data metrics #######
for i in range(6):
    print('-----------------------------')
    print('Threshold = ' + str(THRESHOLDS[i]))
    ds_house_filter = Power(datafilter[i][-1], ds_appliance[-1], ds_status[-1], SEQ_LEN, BORDER, MAX_POWER, False)
    dl_test_filter = DataLoader(dataset = ds_house_filter, batch_size = BATCH_SIZE, shuffle=False)

    print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_dishwasher_network_weights.pth',
              x_test = dl_test_filter, 
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER, thr = 0.5)


dishwasher
F1 score  : 0.823
Precision : 0.790
Recall    : 0.859
Accuracy  : 0.979
MCC       : 0.812
MAE       : 61.91
SAE       : 0.087
-----------------------------
Threshold = 50

dishwasher
F1 score  : 0.821
Precision : 0.786
Recall    : 0.859
Accuracy  : 0.978
MCC       : 0.811
MAE       : 62.18
SAE       : 0.093
-----------------------------
Threshold = 100

dishwasher
F1 score  : 0.820
Precision : 0.783
Recall    : 0.860
Accuracy  : 0.978
MCC       : 0.809
MAE       : 62.39
SAE       : 0.099
-----------------------------
Threshold = 150

dishwasher
F1 score  : 0.817
Precision : 0.778
Recall    : 0.861
Accuracy  : 0.978
MCC       : 0.807
MAE       : 62.70
SAE       : 0.107
-----------------------------
Threshold = 200

dishwasher
F1 score  : 0.817
Precision : 0.775
Recall    : 0.864
Accuracy  : 0.978
MCC       : 0.806
MAE       : 62.97
SAE       : 0.114
-----------------------------
Threshold = 250

dishwasher
F1 score  : 0.817
Precision : 0.775
Recall    : 0.864
Accuracy  : 0.9

## Washing-machine case

In [23]:
APPLIANCE = ['washing_machine']
THRESHOLD = [20.]
MIN_ON = [1600. / 60]
MIN_OFF = [800. / 60]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [24]:
from utils.preprocessing import get_status
ds_meter = []
ds_appliance = []
ds_status = []
houses = [6]

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/washing_machine/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [25]:
from utils.preprocessing import chain2filter

houses = [6]
THRESHOLDS=[50,100,150,200,250,300]    
datafilter=[]
for threshold in THRESHOLDS:
    original = []
    for k in houses:
        ds = pd.read_feather('../data/refit/feather_files/washing_machine/REFIT_%d.feather' %k)
        ds.set_index('datetime', inplace=True)  
        meter = ds[METER]
        original.append(meter)
    datafilter.append(chain2filter(original, threshold, len(houses)))
    
ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/washing_machine/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(len(houses))]

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i], SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]
dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False)

In [28]:
from utils.net import PTPNet
from utils.evaluation import print_metrics

###### print no filtered data metrics ######
print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_washingmachine_network_weights.pth',
              x_test = dl_test,
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER,
              thr = 0.5)
###### print filtered data metrics #######
for i in range(6):
    print('-----------------------------')
    print('Threshold = ' + str(THRESHOLDS[i]))
    ds_house_filter = Power(datafilter[i][-1], ds_appliance[-1], ds_status[-1], SEQ_LEN, BORDER, MAX_POWER, False)
    dl_test_filter = DataLoader(dataset = ds_house_filter, batch_size = BATCH_SIZE, shuffle=False)

    print_metrics(model = PTPNet(1,1,32).cuda(),
              weights_path = '../data/refit/network_weights/REFIT_washingmachine_network_weights.pth',
              x_test = dl_test_filter,
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[-1],
              y_test_stat = ds_status[-1],
              border = BORDER, thr = 0.5)


washing_machine
F1 score  : 0.840
Precision : 0.793
Recall    : 0.894
Accuracy  : 0.995
MCC       : 0.839
MAE       : 8.54
SAE       : 0.128
-----------------------------
Threshold = 50

washing_machine
F1 score  : 0.875
Precision : 0.865
Recall    : 0.886
Accuracy  : 0.996
MCC       : 0.873
MAE       : 7.96
SAE       : 0.025
-----------------------------
Threshold = 100

washing_machine
F1 score  : 0.885
Precision : 0.899
Recall    : 0.872
Accuracy  : 0.996
MCC       : 0.883
MAE       : 7.66
SAE       : -0.029
-----------------------------
Threshold = 150

washing_machine
F1 score  : 0.846
Precision : 0.910
Recall    : 0.791
Accuracy  : 0.995
MCC       : 0.846
MAE       : 7.38
SAE       : -0.131
-----------------------------
Threshold = 200

washing_machine
F1 score  : 0.770
Precision : 0.923
Recall    : 0.661
Accuracy  : 0.994
MCC       : 0.778
MAE       : 6.99
SAE       : -0.284
-----------------------------
Threshold = 250

washing_machine
F1 score  : 0.685
Precision : 0.931
Recal